# Referências da Documentação AWS

- https://docs.aws.amazon.com/nova/latest/userguide/modalities-document.html
- https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management.html
- https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management-create.html

## Dependências

In [ ]:
import json

import boto3
from botocore.exceptions import ClientError

# Constantes

In [ ]:
# ARN do prompt criado no Bedrock Prompt Manager
PROMPT_ARN = "arn:aws:bedrock:us-east-1:123456789012:prompt/ABC123XYZ"

# Configurações do AWS Bedrock

In [ ]:
# Inicializar cliente Bedrock
bedrock_runtime_client = boto3.client('bedrock-runtime')

# Caso de uso

In [ ]:
# Ler documento local
DOCUMENT_PATH = "samples/world_tourism.md"

with open(DOCUMENT_PATH, "rb") as f:
    doc_bytes = f.read()

print(f"Documento lido: {len(doc_bytes)} bytes ({len(doc_bytes) / 1024 / 1024:.2f} MB)")
print(f"Arquivo: {DOCUMENT_PATH}")

In [ ]:
# Variável do prompt: continente para filtrar
prompt_variables = {
    "continent": {"text": "South America"}
}

print(f"Continente selecionado: {prompt_variables['continent']['text']}")

In [ ]:
# Preparar mensagens com anexo do documento
messages = [
    {
        "role": "user",
        "content": [
            {
                "document": {
                    "format": "md",
                    "name": "world-tourism",
                    "source": {"bytes": doc_bytes}
                }
            },
            {"text": "Extract the tourism data from the document."}
        ]
    }
]

print("Mensagem preparada com documento anexado")

In [ ]:
# Executar Chamada ao Bedrock
try:
    # Chamar usando Prompt Manager + Document + Variables
    response = bedrock_runtime_client.converse(
        modelId=PROMPT_ARN,
        messages=messages,
        promptVariables=prompt_variables
    )
    
    # Extrair resultado do Tool Use
    result = response["output"]["message"]["content"][0]["toolUse"]["input"]
    print("Dados extraídos com sucesso!\n")
    print(json.dumps(result, indent=2, ensure_ascii=False))


except ClientError as e:
    error_code = e.response['Error']['Code']
    error_message = e.response['Error']['Message']
    
    print(f"Erro: {error_code}")
    print(f"Mensagem: {error_message}\n")
    print("Sugestões:")
    print("- Verifique se o limite máximo de tokens na configuração do prompt é suficiente (recomendado: 1000+)")
    print("- Verifique se o schema da tool está configurado corretamente no Bedrock Prompt Manager")
    print("- Certifique-se de que o ARN do prompt é válido e acessível")
    print("- Verifique se a variável 'continent' está definida no prompt")

In [ ]:
# Mudar apenas a variável
prompt_variables_europe = {
    "continent": {"text": "Europe"}
}

try:
    print(f"Testando com continente: {prompt_variables_europe['continent']['text']}\n")
    
    # Mesma chamada, apenas mudando a variável
    response = bedrock_runtime_client.converse(
        modelId=PROMPT_ARN,
        messages=messages,
        promptVariables=prompt_variables_europe
    )
    result = response["output"]["message"]["content"][0]["toolUse"]["input"]
    print("Dados da Europa extraídos com sucesso!\n")
    print(json.dumps(result, indent=2, ensure_ascii=False))

except ClientError as e:
    print(f"Erro: {e.response['Error']['Code']}")
    print(f"Mensagem: {e.response['Error']['Message']}")